In [ ]:
from bedrock_agentcore_starter_toolkit.operations.memory.manager import Memory, MemoryManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies import (
    ConsolidationConfig,
    CustomSemanticStrategy,
    CustomSummaryStrategy,
    CustomUserPreferenceStrategy,
    ExtractionConfig,
    InvocationConfig,
    MessageBasedTrigger,
    SelfManagedStrategy,
    SemanticStrategy,
    SummaryStrategy,
    TimeBasedTrigger,
    TokenBasedTrigger,
)

In [2]:
manager = MemoryManager()

✅ MemoryManager initialized for region: None


In [13]:
for memory in manager.list_memories():
    try:
        manager.delete_memory(memory_id=memory.id)
        pass
    except Exception:
        pass
    try:
        status = manager.get_memory(memory_id=memory.id).status
    except Exception:
        status = "DELETED"
    print(f"🔍 DEBUG: Memory status {status}")

Deleted memory: CustomerSupportSemantic10-j28CWNGRR7
🔎 Retrieving memory resource with ID: CustomerSupportSemantic10-j28CWNGRR7...
  ✅ Found memory: CustomerSupportSemantic10-j28CWNGRR7


🔍 DEBUG: Memory status DELETING


Deleted memory: DemoLongTermMemory1-mIRDJI3wdV
🔎 Retrieving memory resource with ID: DemoLongTermMemory1-mIRDJI3wdV...
  ✅ Found memory: DemoLongTermMemory1-mIRDJI3wdV


🔍 DEBUG: Memory status DELETING


In [ ]:
memory = manager.get_or_create_memory(
    name="qDemoMemory",
    strategies=[
        SemanticStrategy(
            name="SemanticStrategy",
            namespaces=["/strategies/{memoryStrategyId}/actors/{actorId}"],
        )
    ],
)

Memory already exists. Using existing memory ID: qDemoMemory-RQOZg27WYc
🔎 Retrieving memory resource with ID: qDemoMemory-RQOZg27WYc...
  ✅ Found memory: qDemoMemory-RQOZg27WYc
Existing {'type': 'SEMANTIC', 'name': 'SemanticStrategy', 'description': None, 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}']}
Requested {'type': 'SEMANTIC', 'name': 'SemanticStrategya', 'description': None, 'namespaces': []}


ValueError: Strategy mismatch for memory 'qDemoMemory'. Strategy 1 mismatch: name: value mismatch ('SemanticStrategy' vs 'SemanticStrategya') Cannot use existing memory with different strategy configuration.

In [8]:
manager.add_summary_strategy(
    memory_id=memory.id,
    name="SemanticStrategy",
    description="Brief description",
)

Updated memory strategies for: DemoMemory-zon5lHECjS


{'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoMemory-zon5lHECjS', 'id': 'DemoMemory-zon5lHECjS', 'name': 'DemoMemory', 'eventExpiryDuration': 90, 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 30, 22, 36, 26, 183000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 22, 36, 28, 864000, tzinfo=tzlocal()), 'strategies': [{'strategyId': 'SemanticStrategy-wEMM9fABMq', 'name': 'SemanticStrategy', 'description': 'Brief description', 'type': 'SUMMARIZATION', 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}'], 'createdAt': datetime.datetime(2025, 9, 30, 22, 36, 29, 33000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 22, 36, 29, 33000, tzinfo=tzlocal()), 'status': 'CREATING'}]}

In [ ]:
print("🔍 DEBUG: Starting control plane operations...")

# Memory creation using strategies with data classes
memory1: Memory = manager.get_or_create_memory(
    name="LargeDemoLongTermMemory",
    description="A temporary memory for short-lived conversations.",
    memory_execution_role_arn="arn:aws:iam::328307993871:role/AgentCoreMemoryTestRole-24edafc2",
    strategies=[
        CustomSemanticStrategy(
            name="SemanticStrategy",
            description="A strategy for semantic understanding.",
            extraction_config=ExtractionConfig(
                append_to_prompt="Extract insights", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
            ),
            consolidation_config=ConsolidationConfig(
                append_to_prompt="Consolidate semantic insights", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
            ),
            namespaces=["support/user/{actorId}/{sessionId}"],
        ),
        CustomSummaryStrategy(
            name="SummaryStrategy",
            description="A strategy for summarizing the conversation.",
            consolidation_config=ConsolidationConfig(
                append_to_prompt="Summarize conversation highlights", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
            ),
            namespaces=["support/user/{actorId}/{sessionId}"],
        ),
        CustomUserPreferenceStrategy(
            name="UserPreferenceStrategy",
            description="A strategy for user preference.",
            extraction_config=ExtractionConfig(
                append_to_prompt="Extract insights", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
            ),
            consolidation_config=ConsolidationConfig(
                append_to_prompt="Consolidate user preferences", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
            ),
            namespaces=["/strategies/{memoryStrategyId}/actors/{actorId}"],
        ),
    ],
)
print("🔍 DEBUG: long-term memory created successfully")
memory1

Memory already exists. Using existing memory ID: LargeDemoLongTermMemory-wKcX9pCmQV
🔎 Retrieving memory resource with ID: LargeDemoLongTermMemory-wKcX9pCmQV...


🔍 DEBUG: Starting control plane operations...


  ✅ Found memory: LargeDemoLongTermMemory-wKcX9pCmQV


ValueError: Strategy mismatch for memory 'LargeDemoLongTermMemory'. Strategy count mismatch. Existing memory has 3 strategies: ['CUSTOM', 'CUSTOM', 'CUSTOM'], but 4 strategies were requested: ['CUSTOM', 'CUSTOM', 'CUSTOM', 'SUMMARIZATION']. Cannot use existing memory with different strategy configuration.

In [5]:
# Memory creation using strategies without data classes
memory2: Memory = manager.get_or_create_memory(
    name="DemoLongTermMemory2",
    description="A temporary memory for long-lived conversations.",
    strategies=[
        {
            "semanticMemoryStrategy": {
                "name": "SemanticStrategy",
                "description": "A strategy for semantic understanding.",
                "namespaces": ["support/user/{actorId}/{sessionId}"],
            }
        }
    ],
)
print("🔍 DEBUG: Long-term memory created successfully")
memory2

Memory already exists. Using existing memory ID: DemoLongTermMemory2-ZsKBvw2Oqs
🔎 Retrieving memory resource with ID: DemoLongTermMemory2-ZsKBvw2Oqs...
  ✅ Found memory: DemoLongTermMemory2-ZsKBvw2Oqs
Existing {'type': 'SEMANTIC', 'name': 'SemanticStrategy', 'description': 'A strategy for semantic understanding.', 'namespaces': ['support/user/{actorId}/{sessionId}']}
Requested {'type': 'SEMANTIC', 'name': 'SemanticStrategy', 'description': 'A strategy for semantic understanding.', 'namespaces': ['support/user/{actorId}/{sessionId}']}
Universal strategy validation passed for memory DemoLongTermMemory2. Strategies match: [SEMANTIC]


🔍 DEBUG: Long-term memory created successfully


{'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory2-ZsKBvw2Oqs', 'id': 'DemoLongTermMemory2-ZsKBvw2Oqs', 'name': 'DemoLongTermMemory2', 'description': 'A temporary memory for long-lived conversations.', 'eventExpiryDuration': 90, 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 30, 1, 10, 8, 881000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 10, 9, 61000, tzinfo=tzlocal()), 'strategies': [{'strategyId': 'SemanticStrategy-NaqKkU3ZpK', 'name': 'SemanticStrategy', 'description': 'A strategy for semantic understanding.', 'type': 'SEMANTIC', 'namespaces': ['support/user/{actorId}/{sessionId}'], 'createdAt': datetime.datetime(2025, 9, 30, 1, 10, 8, 881000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 10, 9, 61000, tzinfo=tzlocal()), 'status': 'ACTIVE'}]}

In [6]:
# Memory creation using strategies without data classes
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.user_preference import UserPreferenceStrategy

memory3: Memory = manager.get_or_create_memory(
    name="DemoLongTermMemory3",
    description="A temporary memory for long-lived conversations.",
    strategies=[
        UserPreferenceStrategy(
            name="UserPreferenceStrategy",
            description="A strategy for user preference.",
            namespaces=["/strategies/{memoryStrategyId}/actors/{actorId}"],
        )
    ],
)
print("🔍 DEBUG: Long-term memory created successfully")
memory3

Memory already exists. Using existing memory ID: DemoLongTermMemory3-EQGD6pFqrh
🔎 Retrieving memory resource with ID: DemoLongTermMemory3-EQGD6pFqrh...
  ✅ Found memory: DemoLongTermMemory3-EQGD6pFqrh
Existing {'type': 'USER_PREFERENCE', 'name': 'UserPreferenceStrategy', 'description': 'A strategy for user preference.', 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}']}
Requested {'type': 'USER_PREFERENCE', 'name': 'UserPreferenceStrategy', 'description': 'A strategy for user preference.', 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}']}
Universal strategy validation passed for memory DemoLongTermMemory3. Strategies match: [USER_PREFERENCE]


🔍 DEBUG: Long-term memory created successfully


{'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory3-EQGD6pFqrh', 'id': 'DemoLongTermMemory3-EQGD6pFqrh', 'name': 'DemoLongTermMemory3', 'description': 'A temporary memory for long-lived conversations.', 'eventExpiryDuration': 90, 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 30, 1, 13, 7, 602000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 13, 8, 134000, tzinfo=tzlocal()), 'strategies': [{'strategyId': 'UserPreferenceStrategy-swPkj5ELUU', 'name': 'UserPreferenceStrategy', 'description': 'A strategy for user preference.', 'type': 'USER_PREFERENCE', 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}'], 'createdAt': datetime.datetime(2025, 9, 30, 1, 13, 7, 604000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 13, 8, 134000, tzinfo=tzlocal()), 'status': 'ACTIVE'}]}

In [7]:
# Confirm server default namespaces are applied
manager.get_memory_strategies(memory_id=memory3.id)

[{'strategyId': 'UserPreferenceStrategy-swPkj5ELUU', 'name': 'UserPreferenceStrategy', 'description': 'A strategy for user preference.', 'type': 'USER_PREFERENCE', 'namespaces': ['/strategies/{memoryStrategyId}/actors/{actorId}'], 'createdAt': datetime.datetime(2025, 9, 30, 1, 13, 7, 604000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 13, 8, 134000, tzinfo=tzlocal()), 'status': 'ACTIVE'}]

In [ ]:
# Memory creation using self managed strategy

# Create SelfManagedStrategy
strategy = SelfManagedStrategy(
    name="DemoMySelfManagedStrategy",
    description="Custom self-managed memory processing",
    trigger_conditions=[
        MessageBasedTrigger(message_count=10),
        TokenBasedTrigger(token_count=8000),
        TimeBasedTrigger(idle_session_timeout=40),
    ],
    invocation_config=InvocationConfig(topic_arn="", payload_delivery_bucket_name=""),
    historical_context_window_size=6,
)

memory4: Memory = manager.get_or_create_memory(
    name="DemoSelfManagedMemory",
    strategies=[strategy],
    description="Memory with self-managed processing",
    memory_execution_role_arn="",
)

print("🔍 DEBUG: Long-term memory created successfully")
memory4

In [8]:
# List all memories
for memory_summary in manager.list_memories():
    print(f"🔍 DEBUG: Memory found : {memory_summary}")

🔍 DEBUG: Memory found : {'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory2-ZsKBvw2Oqs', 'id': 'DemoLongTermMemory2-ZsKBvw2Oqs', 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 30, 1, 10, 8, 881000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 10, 9, 61000, tzinfo=tzlocal()), 'memoryId': 'DemoLongTermMemory2-ZsKBvw2Oqs'}
🔍 DEBUG: Memory found : {'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory3-EQGD6pFqrh', 'id': 'DemoLongTermMemory3-EQGD6pFqrh', 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 30, 1, 13, 7, 602000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 13, 8, 134000, tzinfo=tzlocal()), 'memoryId': 'DemoLongTermMemory3-EQGD6pFqrh'}
🔍 DEBUG: Memory found : {'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/LargeDemoLongTermMemory-wKcX9pCmQV', 'id': 'LargeDemoLongTermMemory-wKcX9pCmQV', 'status': 'ACTIVE', 'createdAt': datetime.datetime(

In [9]:
# Get all the memory strategies available
strategies = manager.get_memory_strategies(memory_id=memory1.id)
strategies

[{'strategyId': 'SemanticStrategy-yavcUMBQKA', 'name': 'SemanticStrategy', 'description': 'A strategy for semantic understanding.', 'configuration': {'type': 'SEMANTIC_OVERRIDE', 'extraction': {'customExtractionConfiguration': {'semanticExtractionOverride': {'appendToPrompt': 'Extract insights', 'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0'}}}, 'consolidation': {'customConsolidationConfiguration': {'semanticConsolidationOverride': {'appendToPrompt': 'Consolidate semantic insights', 'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0'}}}}, 'type': 'CUSTOM', 'namespaces': ['support/user/{actorId}/{sessionId}'], 'createdAt': datetime.datetime(2025, 9, 30, 1, 7, 13, 810000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 7, 14, 28000, tzinfo=tzlocal()), 'status': 'ACTIVE'},
 {'strategyId': 'SummaryStrategy-bJtrdBGkWN', 'name': 'SummaryStrategy', 'description': 'A strategy for summarizing the conversation.', 'configuration': {'type': 'SUMMARY_OVERRIDE', 'consolidati

In [10]:
if "SummaryStrategy" not in [strategy.name for strategy in strategies]:
    manager.add_strategy_and_wait(
        memory_id=memory2.id,
        strategy=SummaryStrategy(
            name="SummaryStrategy",
            description="A strategy for summarizing the conversation.",
            namespaces=["support/user/{actorId}/{sessionId}"],
        ),
    )
    print("🔍 DEBUG: Summary strategy added successfully")
else:
    print("🔍 DEBUG: Summary strategy already exists - skipping memory update")

🔍 DEBUG: Summary strategy already exists - skipping memory update


In [11]:
# Using direct access capability to show memory fields
memory1.description

'A temporary memory for short-lived conversations.'

In [12]:
# Fetch the memory again to see the updated strategies
get_response = manager.get_memory(memory_id=memory1.id)

🔎 Retrieving memory resource with ID: LargeDemoLongTermMemory-wKcX9pCmQV...
  ✅ Found memory: LargeDemoLongTermMemory-wKcX9pCmQV


In [13]:
get_response.status

'ACTIVE'

In [14]:
manager.get_memory_strategies(memory_id=memory1.id)

[{'strategyId': 'SemanticStrategy-yavcUMBQKA', 'name': 'SemanticStrategy', 'description': 'A strategy for semantic understanding.', 'configuration': {'type': 'SEMANTIC_OVERRIDE', 'extraction': {'customExtractionConfiguration': {'semanticExtractionOverride': {'appendToPrompt': 'Extract insights', 'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0'}}}, 'consolidation': {'customConsolidationConfiguration': {'semanticConsolidationOverride': {'appendToPrompt': 'Consolidate semantic insights', 'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0'}}}}, 'type': 'CUSTOM', 'namespaces': ['support/user/{actorId}/{sessionId}'], 'createdAt': datetime.datetime(2025, 9, 30, 1, 7, 13, 810000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 30, 1, 7, 14, 28000, tzinfo=tzlocal()), 'status': 'ACTIVE'},
 {'strategyId': 'SummaryStrategy-bJtrdBGkWN', 'name': 'SummaryStrategy', 'description': 'A strategy for summarizing the conversation.', 'configuration': {'type': 'SUMMARY_OVERRIDE', 'consolidati

In [15]:
for memory in manager.list_memories():
    print(memory.status)

ACTIVE
ACTIVE
ACTIVE


In [16]:
[strategy.strategyId for strategy in manager.get_memory_strategies(memory_id=memory1.id)]

['SemanticStrategy-yavcUMBQKA',
 'SummaryStrategy-bJtrdBGkWN',
 'UserPreferenceStrategy-IPfICy7X1r']

In [17]:
# try:
#     for memory in manager.list_memories():
#         manager.delete_memory(memory_id=memory.id)
# except Exception as e:
#     print(f"🔍 DEBUG: Error deleting memory: {e}")
#     pass

In [18]:
try:
    for memory in manager.list_memories():
        memory = manager.get_memory(memory_id=memory.id)
        print(f"🔍 DEBUG: Memory found with status: {memory.status}")
except Exception as e:
    print(f"🔍 DEBUG: Memory deletion confirmed. Error: {e}")

🔎 Retrieving memory resource with ID: DemoLongTermMemory2-ZsKBvw2Oqs...
  ✅ Found memory: DemoLongTermMemory2-ZsKBvw2Oqs
🔎 Retrieving memory resource with ID: DemoLongTermMemory3-EQGD6pFqrh...
  ✅ Found memory: DemoLongTermMemory3-EQGD6pFqrh
🔎 Retrieving memory resource with ID: LargeDemoLongTermMemory-wKcX9pCmQV...


🔍 DEBUG: Memory found with status: ACTIVE
🔍 DEBUG: Memory found with status: ACTIVE


  ✅ Found memory: LargeDemoLongTermMemory-wKcX9pCmQV


🔍 DEBUG: Memory found with status: ACTIVE
